# SMS Spam Filter Project
## Creating a SMS spam filter using the multinomial Naive Bayes algorithm

Text (SMS) message spam is a serious issue for millions of consumers around the world. While most messages are simply annoying, many spam messages are targeted campaings to steal consumers information. Filtering messages is not a simple endeavour. How does a computer decide whether an imcoming message is spam or legitimate? One method, the choice for this project, is to use the multinomial Naive Bayes algorithm. In general, the algorithm works as such:
1. Learns how humans classify messages.
2. Uses that human knowledge to estimate probabilities for new messages (using Bayes Theorem) as spam or not spam.
3. Classifies a new message based on these probabilities. If the probability a message is spam is higher than non-spam, it will classify the message as spam, and vice-versa. If the probabilities are equal, then we require a human to decide the message classification.

Our goal for this project is:
* To build a SMS spam filter with at least an 80% accuracy using the multinomial Naive Bayes algorithm. 

We will use a dataset of 5,572 already classified SMS messages from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) to "teach" the computer how to classify messages. 

To begin our project, we will import the dataset and explore to familiarize ourselves with these data. 

In [234]:
# Import libraries and dataset
import pandas as pd

sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

In [235]:
sms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [236]:
# Find percentage of spam vs ham (non-spam) messages
sms['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

### Establishing a training and testing set
On a simple exploration of our dataset, we see that about 87% of these messages are ham (non-spam) and 13% are spam. Next, we want to begin building our spam filter. However, before we start creating the software, we want to establish a method to test and verify the software works correctly. If we wait until the end of the project, we might be tempted to create a biased test so the software passes. 

When we complete the spam filter, we need to test how well it classifies new messages. In order to do this, we will first split our data into two sets:
* A training set: we will use this to "train" the algorithm how to classify messages
* A test set: we will use this to test the accuracy of the filter. 

In general, we want to use as much data as possible to train the algorithm while having enough data to test the algorithm. We will keep 80% of the dataset for training and 20% for testing. 
* The training set will have 4,458 messages
* The test set will have 1,114 messages

The test is simple: as the messages are already classified by a human, we only need to compare the classifications the algorithm makes to the human classifications. We will use this test once we actually build the software, but first we will split the dataset and begin creating the algorithm.

In [237]:
# First we will randomize the dataset
random_sms = sms.sample(frac=1, random_state=1) # random_stat=1 for consistency

# Split dataset into train and test
train = random_sms.iloc[:4458, :].reset_index()
test = random_sms.iloc[4458:, :].reset_index()

In [238]:
# Verify sample is consisten with entire dataset
train['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [239]:
# Verify sample is consisten with entire dataset
test['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

### Data Cleaning
Before we can move on to creating the algorithm, we need to clean these datasets. The goal for this cleaning process is to transform the train and test dataframes into dataframes with columns for each word in the entire vocabulary of words in these messages. Each row will still represent a message, but instead of an `SMS` column containing a string with the message, it will contain the number of times that word occurs in a given message. In other words, each column will contan the frequency for that word in the message. 

The SMS messages contain capitalizations and punctuation marks that we do not want. In order to transform the dataframes, we will first remove punctuation and make all the messages lowercase. 

In [240]:
# Remove punctuations from train
train['SMS'] = train['SMS'].str.replace('\W', ' ', regex=True)

In [241]:
# Make lowercase train
train['SMS'] = train['SMS'].str.lower()

Next, we will create a list called `vocabulary` that contains all the unique words that occur in these messages. We will first split the strings in `SMS` into lists. Then, we will add all the individual words to the list `vocabulary`. Finally, we will conver the list to a set and back to remove duplicate words. 

In [242]:
# Transform each row in `SMS` into a list
train['SMS'] = train["SMS"].str.split()

# Initialize list for vocabulary
vocabulary = []

# Iterate over `SMS` and add each word to `vocabulary`
for msg in train['SMS']:
    for word in msg:
        vocabulary.append(word)

# Keep only unique words
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)


Next, we want to create a dictionary we can use to create a new dataframe with the columns representing all words in `vocabulary` and the rows indicating the number of times each word occurs in an individual message. 

* First, we will start by creating a dictionary `word_counts_per_sms` where each key is a unique word in `vocabulary` and each index is a list of zeros equal in length to the length of the training set, `train['SMS']`. 
* Next, we will loop over `train['SMS']` using the `enumerate()` function to get both the index and the SMS message. 
    * Using a nested loop, we loop over `sms` (where `sms` is a list of strings, where each sstring represents a word in a message).
        * Inside that loop, we increment `word_counts_per_sms[word][index]` by 1.

In [243]:
# Define dictionary word_counts_per_sms with each key representing a word from vocabulary and each index a list of 0s of len(train['SMS'])
word_counts_per_sms = {
    unique_word: [0] * len(train["SMS"]) for unique_word in vocabulary
}

# Iterate over train["SMS"] using enumerate() to add counts to each key in word_counts_per_sms
for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1



In [244]:
# Transform word_counts_per_sms to dataframe
word_counts = pd.DataFrame(word_counts_per_sms)

# Concatenate word_counts with train
train_updated = pd.concat([train, word_counts], axis=1)

### Building the Spam Filter
Now, that we have a training set to work with, we can begin building the SMS Spam filter. The Naive Bayes Algorithm needs to calculate the probabilities of a given message being spam or not. Bayes Theorem gives us the following fomulae for probability:

$$
P(Spam|w_{1}, w_{2}, ..., w_{n}) \space \alpha \space P(Spam) \space • \space \prod_{i=1}^{n} P(w_{i}|Spam)
$$(eq1)

$$
P(Ham|w_{1}, w_{2}, ..., w_{n}) \space \alpha \space P(Ham) \space • \space \prod_{i=1}^{n} P(w_{i}|Ham)
$$(eq2)
where  $w_{1}, w_{2}, ..., w_{n}$ is a given word in an incoming message. To calculate $P(w_{i}|Spam)$ and $P(w_{i}|Ham)$, we need to use the equations:

$$
P(w_{i}|Spam) = \frac{N_{w_{i}|Spam} + \alpha}{N_{Spam} + \alpha \space • \space N_{Vocabulary}}
$$

$$
P(w_{i}|Ham) = \frac{N_{w_{i}|Ham} + \alpha}{N_{Ham} + \alpha \space • \space N_{Vocabulary}}
$$

First, we will calculate our constants for the above equations:
* P(Spam) and P(Ham)
* $N_{Spam}$, $N_{Ham}$, and $N_{Vocabulary}$

We will keep in mind that:
* $N_{Spam}$ is the number of words in all spam messages. It is not the number of spam messages, or the number of unique words in spam messages.
* $N_{Ham}$ is the number of words in all non-spam messages. It is not the number of non-spam messages, or the number of unique words in non-spam messages.

Furthermore, we will use Laplace Smoothing and set $\alpha = 1$. 

In [245]:
# Calculate p_spam, p_ham, n_vocabulary, n_spam, n_ham
p_spam = train_updated['Label'].value_counts(normalize=True)['spam'] # Probability of message being spam
p_ham = train_updated['Label'].value_counts(normalize=True)['ham'] # Probability of message being non-spam

n_vocabulary = len(vocabulary) # Number of unique words in the vocabulary
n_ham = train_updated[train_updated['Label'] == 'ham'].iloc[:, 3:].sum().sum() # Number of words in non-spam messages
n_spam = train_updated[train_updated['Label'] == 'spam'].iloc[:, 3:].sum().sum() # Number of words in spam messages

alpha = 1 # Smoothing parameter

Now that we have calculated the constant values needed for the algorithm, we will begin calculated probabilities of individual words in our vocabulary. In order to calculate the probabilities for incoming messages, we need to calculate $P(w_{i}|Spam)$ and $P(w_{i}|Ham)$, which will vary for each individual word. 

While $P(w_{i}|Spam)$ and $P(w_{i}|Ham)$ are different for each word, the probability for each individual word is constant. We need only to calculate $P(w_{i}|Spam)$ and $P(w_{i}|Ham)$ once for every word in our vocabulary. Then, the resulting probabilities can be used to classify messages. Because of this, the Naive Bayes Algorithm is very fast. Only a small number of calculations need to be performed for any new message. 

Next, we will create two dictionaries where each key is a unique word from `vocabulary` and each value is $P(w_{i}|Spam)$ or $P(w_{i}|Ham)$ for the given word. We will use the formiulae:
$$
P(w_{i}|Spam) = \frac{N_{w_{i}|Spam} + \alpha}{N_{Spam} + \alpha \space • \space N_{Vocabulary}}
$$

$$
P(w_{i}|Ham) = \frac{N_{w_{i}|Ham} + \alpha}{N_{Ham} + \alpha \space • \space N_{Vocabulary}}
$$
to calculate the probabilities for each word. 

In [246]:
# Dictionary for P(w_i|Spam)
p_w_i_spam = {
    unique_word: 0 for unique_word in vocabulary
}

# Dictionary for P(w_i|Ham)
p_w_i_ham = {
    unique_word: 0 for unique_word in vocabulary
}

In [247]:
# Isolate Spam and Ham messages
spam_messages = train_updated.query("Label == 'spam'")
ham_messages = train_updated.query("Label == 'ham'")

In [248]:
# Calculate probabilities for each word in vocabulary

for word in vocabulary:
    n_w_spam = spam_messages[word].sum()
    n_w_ham = ham_messages[word].sum()
    p_w_spam = (n_w_spam + alpha) / (n_spam + (alpha * n_vocabulary))
    p_w_ham = (n_w_ham + alpha) / (n_ham + (alpha * n_vocabulary))
    p_w_i_spam[word] = p_w_spam
    p_w_i_ham[word] = p_w_ham



Now that we have the probabilities we need, we will create a function `classify()` that takes a message as input and returns the classification as `spam` or `ham`. We will use the formulae
$$
P(Spam|w_{1}, w_{2}, ..., w_{n}) \space \alpha \space P(Spam) \space • \space \prod_{i=1}^{n} P(w_{i}|Spam)
$$(eq1)

$$
P(Ham|w_{1}, w_{2}, ..., w_{n}) \space \alpha \space P(Ham) \space • \space \prod_{i=1}^{n} P(w_{i}|Ham)
$$(eq2)
where  $w_{1}, w_{2}, ..., w_{n}$ is a given word in an incoming message
to calculate the probabilities for each message. If $P(Spam|w_{1}, w_{2}, ..., w_{n})$ is higher than $P(Ham|w_{1}, w_{2}, ..., w_{n})$ then the message is classified as `spam`. Otherwise, if the opposite is true, the message will be classified as `ham`. If the probabilities are equivalent, a human is required to classify the message.

In [249]:
# Define classify(message)
import re

def classify(message):
    # Input message assumed to be a string.
    # Strip punctuation and make lowercase. Then, split into list of strings.
    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # Initialize probabilities with p_spam and p_ham (In accordance with Bayes Theorem)
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    # Iterate over message and calculate probabilities of spam and ham
    for word in message:
        if word in p_w_i_spam:
            p_spam_given_message *= p_w_i_spam[word]
        if word in p_w_i_ham:
            p_ham_given_message *= p_w_i_ham[word]


    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [250]:
# Test
classify('WINNER!! This is the secret code to unlock the money: C3421.')
classify("Sounds good, Tom, then see u there")

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam
P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


### Testing the algorithm
Now that we have completed the `classify()` function, we can test it on the `test` DataFrame we split from the original dataset. We will compare the output of our algorithm with the human classifications in the `test` set to verify the accuracy of our work. 

First, we will modify the `classify()` function to return `ham`, `spam`, or `needs human classification` instead of printing the results. Then we will create a new column in the `test` DataFrame and apply the function to the `SMS` column. We will then calculate the accuracy of the algorithm:
$$
Accuracy \space = \space \frac{number \space of \space correctly \space classified \space messages}{total \space number \space of \space classified \space messages}
$$

In [251]:
# Create new function classify_test_set(message)

def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in p_w_i_spam:
            p_spam_given_message *= p_w_i_spam[word]

        if word in p_w_i_ham:
            p_ham_given_message *= p_w_i_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [252]:
# Create predicted column in test
test['predicted'] = test['SMS'].apply(classify_test_set)
test.head()

,index,Label,SMS,predicted
0,2131,ham,Later i guess. I needa do mcat study too.,ham
1,3418,ham,But i haf enuff space got like 4 mb...,ham
2,3424,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,1538,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,5393,ham,"All done, all handed in. Don't know if mega sh...",ham


In [253]:
# Calculate accuracy
test['correct'] = test['Label'] == test['predicted']
print('Accuracy of filter is: ', (test['correct'].sum() / test.shape[0]))

Accuracy of filter is:  0.9874326750448833


Our spam filter has performed very well, having a 98.7% accuracy for classifying 1,114 new messages.